In [2]:
import psycopg2
import basic_system_data
from pandas import read_sql_query
import geopandas as gpd
import pyproj
from pylab import *
import pdb

#this reads the counts for each route, and distributes them as counts on their constituent road link segments

con = psycopg2.connect(database=basic_system_data.dbname, user=basic_system_data.username)
cur = con.cursor()
wgs84  = pyproj.Proj(init='epsg:4326')  #WGS84
bng    = pyproj.Proj(init='epsg:27700') #british national grid
sql = "SELECT * FROM routecount, route WHERE routecount.routeid=route.routeid AND route.originsiteid != '';" #only get good routes
df_rc = read_sql_query(sql,con)
for i in range(0,df_rc.shape[0]): 
    rc=df_rc.iloc[i]
    sql = "SELECT * FROM routelink WHERE routeid='%s'"%rc['routeid'][0]     #get cached list of gid links in this route
    df_links = read_sql_query(sql,con)
    count = rc['count']
    winlenseconds = rc['winlenseconds']
    for j in range(0,df_links.shape[0]):
        timestamp = rc['timestamp'] #TODO  should estimate what time this link was reached - hacking here
        link = df_links.iloc[j]                   #route component
        gid_link = link['link_gid']
        sql = "INSERT INTO linkcount (gid, timestamp, winlenseconds, count) VALUES ('%s', '%s', %d, %d);"%(gid_link, timestamp, winlenseconds, count)
        print sql
        cur.execute(sql)
        con.commit()

